## Setup

In [1]:
%pip install ipympl
%pip install python-dotenv
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import random
import wandb

# Data

## Landmark data

In [3]:
dataset_path = '/exchange/dspro2/silent-speech/ASL_Landmarks_Dataset/'
#'/exchange/dspro2/silent-speech/full-dataset/'

# Full dataset
#X = np.load(dataset_path + "X_landmarks.npy")
#y = np.load(dataset_path + "y_labels.npy") #['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R''S' 'Space' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 #'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 'u' 'v' 'w' 'x' 'y' 'z']

# Split data for comparison with other models
X_train = np.load(dataset_path + "Train/X_train.npy")
X_val = np.load(dataset_path + "Validation/X_val.npy")
X_test = np.load(dataset_path + "Test/X_test.npy")
y_train = np.load(dataset_path + "Train/y_train.npy")
y_val = np.load(dataset_path + "Validation/y_val.npy")
y_test = np.load(dataset_path + "Test/y_test.npy")

y = np.array([label.upper() for label in y])
print(f'X shape: {X.shape}')
print(f'X_train shape {X_train.shape}')
print(f'X_validation shape {X_val.shape}')
print(f'X_test shape {X_test.shape}')
assert ((X_train.shape[0] + X_val.shape[0] + X_test.shape[0]) == X.shape[0]), "Mismatch between X length and X splits length"

print(f'y[0] {y[0]}, y shape {y.shape}')
print(f'y_train shape {y_train.shape}')
print(f'y_val {y_val.shape}')
print(f'y_test {y_test.shape}')
assert ((y_train.shape[0]+ y_val.shape[0] + y_test.shape[0]) == y.shape[0]), "Mismatch between X length and X splits length"

FileNotFoundError: [Errno 2] No such file or directory: '/exchange/dspro2/silent-speech/ASL_Landmarks_Dataset/Train/X_train.npy'

### Visualize data

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
class_counts = pd.Series(y_train).value_counts().sort_index()

plt.figure(figsize=(12, 5))
sns.barplot(x=class_counts.index, y=class_counts.values, color='blue')#, palette="viridis")
plt.xlabel("Letters")
plt.ylabel("Count")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.title("Classes' distribution on train set")
plt.xticks(rotation=45)  # Rotate labels if crowded
plt.show()

assert (class_counts.sum() == y_train.shape), "class_counts does not correspond to y_train length"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Normalize the landmarks (Standardization: mean=0, std=1)
# Ignores the 3D structure. To let go.
scaler = StandardScaler()
X_train_norm1 = scaler.fit_transform(X_train)  # No reshape needed
X_val_norm1 = scaler.transform(X_val)
X_test_norm1 = scaler.transform(X_test)


# Fit scaler on all x/y/z coordinates, by axis.
# Good for outliers
scaler = StandardScaler()
# Reshape to (n_samples * 21 poses, 3 coordinates)
X_train_reshaped = X_train.reshape(-1, 3)
X_train_scaled = scaler.fit_transform(X_train_reshaped)
# Reshape back to original (n_samples, 63) format
X_train_norm2 = X_train_scaled.reshape(X_train.shape)

X_val_reshaped = X_val.reshape(-1, 3)
#No fit transform to avoid bias the validation
# TODO CHECK
X_val_scaled = scaler.fit_transform(X_val_reshaped)
# Reshape back to original (n_samples, 63) format
X_val_norm2 = X_val_scaled.reshape(X_val.shape)


# To normalize from different distances, avoids size bias
# A utiliser
# Reshape to (n_samples * 21 poses, 3 coordinates)
X_train_reshaped = X_train.reshape(-1, 3)
# Center to origin
X_train_centered = X_train_reshaped - X_train_reshaped.mean(axis=0)
# Scale to unit sphere
max_train_distance = np.max(np.linalg.norm(X_train_centered, axis=1))
X_train_normalized = X_train_centered / max_train_distance
# Reshape back
X_train_norm3 = X_train_normalized.reshape(X_train.shape)


X_val_reshaped = X_val.reshape(-1, 3)
# Center to origin
X_val_centered = X_val_reshaped - X_val_reshaped.mean(axis=0)
# Scale to unit sphere
X_val_normalized = X_val_centered / max_train_distance
# Reshape back
X_val_norm3 = X_val_normalized.reshape(X_val.shape)


# Set up the figure and axes
fig, axes = plt.subplots(2, 2, figsize=(18, 6))  # 2 rows, 2 columns, size 18x6 for the entire figure

# Plotting for each subplot (example with boxplots)
sns.boxplot(data=X_train, ax=axes[0, 0])  # Top-left subplot
axes[0, 0].set_title('Standardization z-score')

sns.boxplot(data=X_train_norm1, ax=axes[0, 1])  # Top-right subplot
axes[0, 1].set_title('Normalization by axis')

sns.boxplot(data=X_train_norm2, ax=axes[1, 0])  # Bottom-left subplot
axes[1, 0].set_title('Against outliers')

sns.boxplot(data=X_train_norm3, ax=axes[1, 1])  # Bottom-right subplot
axes[1, 1].set_title('For 3D poses')

# Adjust the layout to avoid overlap
plt.tight_layout()

# Show the figure
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm # Color map
# 63 hand poses, 21 points/pose, 3 coordinates per point
X_train_samples = X_train[:10]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(projection='3d')
n = 111

# Color Map
cmap = plt.get_cmap()

for index, pose in enumerate(X_train_samples):
    # Reshape in 21x3
    points = pose.reshape(-1,3)
    x, y, z = points[:, 0], points[:,1], points[:,2]
    color = cmap(index / len(X_train_samples)) # NOrmalize index to [0,1]
    ax.scatter(x,y,z, marker='o', color=color)
    #for i in range(len(x)-1):
    #    ax.plot([x[i], x[i+1]], 
    #            [y[i], y[i+1]],
    #            [z[i], z[i+1]], color=color)
    
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Hand poses')
plt.show()

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm # Color map
# 63 hand poses, 21 points/pose, 3 coordinates per point
X_train_samples = X_train[2:4]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(projection='3d')
n = 111

# Color Map
cmap = plt.get_cmap()

for index, pose in enumerate(X_train_samples):
    # Reshape in 21x3
    points = pose.reshape(-1,3)
    x, y, z = points[:, 0], points[:,1], points[:,2]
    color = cmap(index / len(X_train_samples)) # Normalize index to [0,1]
    ax.scatter(x,y,z, marker='o', color=color)
    for i in range(len(x)-1):
        ax.plot([x[i], x[i+1]], 
                [y[i], y[i+1]],
                [z[i], z[i+1]], color=color, linestyle='-')
    
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_title('Hand poses')

ax.view_init(elev=20, azim=30)
plt.show()

## Data transformation into tensors

In [ ]:
device = torch.device("cuda")
# Transform into tensors
#X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
#y_train = torch.tensor(y_train, dtype=torch.long).to(device)
#X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
#y_val = torch.tensor(y_val, dtype=torch.long).to(device)
#X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
#y_test = torch.tensor(y_test, dtype=torch.long).to(device)

def np_into_tensor_gpu(np_array):
    device = torch.device("cuda")
    np_array = torch.tensor(np_array, dtype=torch.float32).to(device)
    return np_array

## Images data

## CNN Landmarks Model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# Landmarks
class AslCNN_landmarks(nn.Module):
    def __init__(self, num_classes):
        super(AslCNN, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, padding=1) # padding=1
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1) # padding=1
        self.fc1 = nn.Linear(32 * 21, 128) #(32 * 17, 128)  # Adjusted fully connected layer
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = x.squeeze(1)
        x = x.permute(0, 2, 1)  # Reshape from (batch, 21, 3) → (batch, 3, 21)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## CNN images model

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class AslCNN_img(nn.Module):
    def __init__(self, num_classes, kernel_size=3, padding=1, channels=[16,32,64]):
        super(AslCNN_img, self).__init__()

    
        self.conv1 = nn.Conv2d(3, channels[0], kernel_size, padding)
        self.pool = nn.MaxPool2d(2,2)
    
        self.conv2 = nn.Conv2d(channels[0], channels[1], kernel_size, padding)
        self.conv3 = nn.Conv2d(channels[1], channels[2], kernel_size, padding)

        # For 400x400 images size and 3 pooling layers
        final_spatial_dim = 400 // (2**3) 
        flattened_size = channels[2] * final_spatial_dim * final_spatial_dim
    
        self.fc1 = nn.Linear(flattened_size, 128)
        self.fc2 = nn.Linear(128, num_classes) # Number of classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

## Data loading

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
device = torch.device("cuda")

# Create DataLoaders
X_train3 = np_into_tensor_gpu(X_train_norm3)
X_train3 = X_train3.view(-1, 1, 21, 3)  # Adjust this based on expected shape (4D and not 2D)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)

X_val3 = np_into_tensor_gpu(X_val_norm3)
X_val3 = X_val3.view(-1, 1, 21, 3)


train_dataset3 = TensorDataset(X_train3, y_train)
val_dataset3 = TensorDataset(X_val3, y_val)
batch_size = 128
train_loader = DataLoader(train_dataset3, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset3, batch_size=batch_size, shuffle=False)

print(f'X_train_tensor shape: {X_train3.shape}')


## Training, validation, test loops

In [ ]:
"""import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Define Model, Loss, and Optimizer
num_classes = len(torch.unique(y_train_tensor)) # 27
model = AslCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Save metrics for accuracy
train_predictions = []
train_labels = []
train_loss = []

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in train_loader: 
        optimizer.zero_grad()
        outputs = model(X_batch) # goes in the AslCNN
        
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        train_loss.append(total_loss/len(train_loader))
    
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

print("Training complete!")"""

In [ ]:
import datetime
import torch.optim as optim
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score
import wandb 



# Function to train the model
def train(model, train_loader, criterion, optimizer):
    device = torch.device("cuda")
    model.train()
    total_loss = 0
    batch_losses = []

    # Metrics for accuracy per epoch
    train_predictions = []
    train_labels = []

    
    for X_batch, y_batch in train_loader: 

        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Reset gradient
        optimizer.zero_grad()
        
        #Raw outputs from the model
        # Forward pass automatically called by Pytorch
        outputs = model(X_batch)  
        # Loss from outputs        
        loss = criterion(outputs, y_batch) 

        #Backward pass
        loss.backward()  # Backpropagate gradients
        optimizer.step()  # Update model parameters

        #End of training loop
        
        # Log metrics
        total_loss += loss.item()  # Accumulate total loss
        #batch_losses.append(loss.item())  # Store individual batch losses
        
        # Convert outputs to predicted labels (highest logit for each sample)
    # TODO put it before the backprop
        _, predicted = torch.max(outputs, 1)  # Get predicted labels (indices of max logits)

        train_predictions.append(predicted)
        train_labels.append(y_batch)

    # Calculate the average loss for the epoch
    epoch_avg_loss = total_loss / len(train_loader)
    return epoch_avg_loss, train_predictions, train_labels


# Function to validate the model
def validate(model, val_loader, criterion):
    device = torch.device("cuda")
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    val_predictions = []
    val_labels = []
    
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            outputs = model(X_val_batch)
            loss = criterion(outputs, y_val_batch)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_val_batch).sum().item()
            total += y_val_batch.size(0)
            
            # Move tensors to CPU before appending
            val_predictions.append(predicted.cpu())  
            val_labels.append(y_val_batch.cpu())  

    # Convert tensors to numpy arrays after moving to CPU
    val_predictions = torch.cat(val_predictions).cpu().numpy()
    val_labels = torch.cat(val_labels).cpu().numpy()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total  # Calculate accuracy

    # Calculate precision, recall
    precision = precision_score(val_labels, val_predictions, average='weighted', zero_division=0)
    recall = recall_score(val_labels, val_predictions, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(val_labels, val_predictions)

    # Calculate specificity (for binary classification)
    if conf_matrix.shape == (2, 2):
        specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    else:
        specificity = 0  # Not applicable for multi-class
    
    sensitivity = recall  # Equivalent to recall for binary classification

    return avg_val_loss, val_accuracy, precision, recall, specificity, sensitivity, sk_learn_acc



# Main training and validation loop
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda")
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # Save metrics for accuracy per epoch
    epoch_losses = []  # Loss values per epoch
    best_val_accuracy = 0.0
    best_epoch = 0
    best_model_predictions = []
    best_model_labels = []

    for epoch in range(num_epochs):
        # Train the model
        train_avg_loss, train_predictions, train_labels = train(model, train_loader, criterion, optimizer)
        epoch_losses.append(train_avg_loss)
        
        # Log training loss
        wandb.log({
            "train_loss": train_avg_loss
        })
        
        print(f"Epoch {epoch+1}, Train Loss: {train_avg_loss:.4f}")

        # Validate the model
        val_avg_loss, val_accuracy, precision, recall, specificity, sensitivity = validate(model, val_loader, criterion)

        # Log validation metrics
        wandb.log({
            "epoch": epoch +1,
            "val_loss": val_avg_loss,
            "val_accuracy": val_accuracy,
            "precision": precision,
            "recall": recall,
            "specificity": specificity,
            "sensitivity": sensitivity
        })
        
        print(f"Epoch {epoch+1}, Validation Loss: {val_avg_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")
        
        # Save the model with the best validation accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch + 1
            best_model_predictions = train_predictions.copy()
            best_model_labels = train_labels.copy()
            
            # Save the model state (best model weights)
            torch.save(model.state_dict(), f'best_model_{current_time}.pth')

    # Print training results after all epochs
    print("Training complete!")
    print(f"Best model achieved at epoch {best_epoch} with validation accuracy: {best_val_accuracy:.2f}%")
    
    return model, best_model_predictions, best_model_labels, epoch_losses, sk_learn_acc



In [ ]:
# CNN IMAGES

num_classes = 28 
device = torch.device("cuda")
model = AslCNN_img(num_classes=num_classes, kernel_size=3, padding=1, channels=[16,32,64]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001) #weight_decay = regularization
criterion = nn.CrossEntropyLoss()

# Call the train_and_validate function to train and validate the model
trained_model, best_model_predictions, best_model_labels, epoch_losses, sk_learn_acc = train_and_validate(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=2)


In [ ]:
import wandb
# check the .env file 
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get and print the WANDB_API_KEY
wandb_api_key = os.getenv("WANDB_API_KEY_DSPRO02")
#print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")
wandb.login()

In [ ]:
# Initialize wandb logging
run_nr = 1
run = wandb.init(project="ASL_CNN", 
                 name=f"CNN_test_standardization3_{run_nr}",
                 config={"Conv1d": "in=3, out=16, kernel=3, pad=1",
                         "Conv1d_2": "in=16, out=32, kernel=3, pad=1",
                         "fc1": (32*21, 128),
                         "fc2": (128, 27),
                         "activations": "ReLU",
                         "epoch": 10,
                         "batch_size": 1024,
                         "optimizer": "adam",
                         "learning_rate": 1e-3,
                         "criterion": "cross_entropy",
                         "weight_decay": 1e-4,  })

config = wandb.config

num_classes = len(torch.unique(y_train)) # 27
device = torch.device("cuda")
model = AslCNN_landmarks(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) #weight_decay = regularization
criterion = nn.CrossEntropyLoss()

# Call the train_and_validate function to train and validate the model
trained_model, best_model_predictions, best_model_labels, epoch_losses = train_and_validate(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=config.epoch)



In [ ]:
wandb.finish() # Except if testing after

In [ ]:
X_test_tensor = X_test_tensor.view(-1, 1, 21, 3) # already done above

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
wandb.finish()

## Search for data leakage

In [ ]:
# Check if data leakage
from sklearn.model_selection import train_test_split
import numpy as np

# Full dataset tensors
indices = np.arange(len(X))  # Create index list
print(indices)
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

# Convert indices to sets for comparison
train_indices = set(train_idx)
val_indices = set(val_idx)

print("Train and Val Overlap:", train_indices.intersection(val_indices))  # Should be empty


In [ ]:
# Attempt with shuffled y to search bias in the model - can it learn the noise?

from torch.utils.data import TensorDataset, DataLoader
y_shuffled = y_train_tensor[torch.randperm(y_train_tensor.size(0))]  # Shuffle labels

if not isinstance(y_shuffled, torch.Tensor):
    y_shuffled = torch.tensor(y_shuffled, dtype=torch.long).to(device)
    
# Create a new dataset with shuffled labels
shuffled_dataset = TensorDataset(X_train_tensor, y_shuffled)

# Load into DataLoader
shuffled_loader = DataLoader(shuffled_dataset, batch_size=32, shuffle=True)

# Initialize wandb logging
run_nr = 1
run = wandb.init(project="ASL_CNN", 
                 name=f"CNN_random_leakage_search_{run_nr}",
                 config={"Conv1d": "in=3, out=16, kernel=3, pad=1",
                         "Conv1d_2": "in=16, out=32, kernel=3, pad=1",
                         "fc1": (32*21, 128),
                         "fc2": (128, 27),
                         "activations": "ReLU",
                         "epoch": 10,
                         "batch_size": 1024,
                         "optimizer": "adam",
                         "learning_rate": 1e-3,
                         "criterion": "cross_entropy",
                         "weight_decay": 1e-4,  })

config = wandb.config

num_classes = len(torch.unique(y_train_tensor)) 
device = torch.device("cuda")
model = AslCNN(num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) #weight_decay = regularization
criterion = nn.CrossEntropyLoss()

# Call the train_and_validate function to train and validate the model
trained_model, best_model_predictions, best_model_labels, epoch_losses = train_and_validate(
    model, shuffled_loader, val_loader, criterion, optimizer, num_epochs=config.epoch)

In [ ]:
wandb.finish()